# I, Phương pháp 

## Phương pháp 1: MnasNet

**Ý tưởng:**  
MnasNet sử dụng **Reinforcement Learning** với một **controller** sinh ra kiến trúc mạng con $ \mathcal{A} $.  
Mỗi kiến trúc được huấn luyện sơ bộ, đánh giá accuracy và đo **độ trễ thực tế (latency)** trên thiết bị di động.  
Hàm thưởng $R(\mathcal{A})$ kết hợp **độ chính xác** và **độ trễ**:

- Controller sinh chuỗi hành động $ a_1, a_2, \dots, a_T $ mô tả kiến trúc.
- Huấn luyện mô hình tương ứng, thu được accuracy $ \mathrm{ACC}(\mathcal{A}) $ và độ trễ $ \mathrm{LAT}(\mathcal{A}) $.

Hàm thưởng:

$$
R(\mathcal{A}) = \mathrm{ACC}(\mathcal{A}) \times \big(\mathrm{LAT}(\mathcal{A})\big)^\beta
$$

với $\beta < 0$ để phạt các mô hình chậm.  
Controller được tối ưu bằng policy gradient:

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\mathcal{A}\sim \pi_\theta} \big[ R(\mathcal{A}) \, \nabla_\theta \log \pi_\theta(\mathcal{A}) \big]
$$


## Phương pháp 2: MobileNetV3

**Ý tưởng:**  
MobileNetV3 được thiết kế bằng cách **dùng NAS (từ MnasNet) để xác định kiến trúc tổng thể**, sau đó **tinh chỉnh thủ công** để tối ưu thêm.  
Cấu trúc mạng gồm các **Inverted Residual Blocks** kết hợp **Squeeze-and-Excitation (SE)** và **hàm kích hoạt h-swish**.

- Inverted Residual block: giảm số kênh → depthwise conv → tăng số kênh
- SE module: học trọng số kênh bằng cơ chế attention
- Hàm h-swish:

$$
\mathrm{h\!-\!swish}(x) = x \cdot \frac{\mathrm{ReLU6}(x+3)}{6}
$$

MobileNetV3 có hai biến thể:
- **Large:** tối ưu độ chính xác cao
- **Small:** tối ưu độ trễ thấp cho thiết bị nhỏ


## Phương pháp 3: Once-for-All (OFA)

**Ý tưởng:**  
Huấn luyện **một Supernet $ \mathcal{N}_\mathrm{super} $** chứa tất cả kiến trúc con khả dĩ, sau đó **rút trích (subnet)** phù hợp với từng giới hạn phần cứng mà không cần huấn luyện lại.  

- Không gian tìm kiếm: $\mathcal{S} = \{\text{depth}, \text{width}, \text{kernel size}, \text{resolution}\}$
- Huấn luyện theo chiến lược **progressive shrinking**:
  - Bắt đầu train mô hình lớn nhất
  - Dần cho phép các subnet nhỏ hơn dùng chung trọng số

Với một subnet $\mathcal{A} \in \mathcal{S}$:

$$
\theta_\mathcal{A} \subset \theta_\mathrm{super}
$$

và được đánh giá theo:

$$
\max_{\mathcal{A} \in \mathcal{S}} \ \mathrm{ACC}(\mathcal{A}) \quad \text{s.t.} \quad \mathrm{LAT}(\mathcal{A}) \le L_\mathrm{target}
$$

Nhờ weight sharing, tất cả subnet đều dùng chung trọng số đã huấn luyện từ Supernet.


## Phương pháp 4: TinyNAS

**Ý tưởng:**  
TinyNAS hướng tới thiết bị cực nhỏ (IoT), nên cần **NAS cực nhanh và hiệu quả**.  
Nó sử dụng cách tiếp cận **predictor-based evolutionary search** với không gian kiến trúc đã được thu hẹp từ trước.

- Không gian kiến trúc nhỏ: $\mathcal{S} = \{\mathcal{A}_1, \mathcal{A}_2, \dots \}$
- Predictor $ f(\mathcal{A}) $ ước lượng accuracy từ kiến trúc
- Evolutionary search để chọn kiến trúc:

$$
\mathcal{A}^* = \arg\max_{\mathcal{A}\in \mathcal{S}} \ f(\mathcal{A}) \quad \text{s.t.} \quad \mathrm{LAT}(\mathcal{A}) \le L_\mathrm{max}
$$

Predictor được huấn luyện bằng một tập nhỏ các kiến trúc đã train thật (data-efficient).  
Sau đó chỉ train mô hình tốt nhất $ \mathcal{A}^* $ từ đầu để dùng triển khai.


# II, Đánh giá

- **Top-1 / Top-5 Accuracy**  
  - Độ chính xác của mô hình trên tập kiểm tra.  
  - **Top-1 Accuracy**: phần trăm mẫu có nhãn dự đoán đúng nhất khớp với nhãn thật.  
  - **Top-5 Accuracy**: phần trăm mẫu có nhãn đúng nằm trong 5 dự đoán cao nhất.  

- **Model Size (Params)**  
  - Số lượng tham số học của mô hình (thường tính bằng triệu — M).  
  - Chỉ số này ảnh hưởng trực tiếp đến khả năng deploy trên thiết bị giới hạn tài nguyên.

- **FLOPs (Floating Point Operations)**  
  - Số phép tính dấu phẩy động cần thực hiện để suy luận 1 ảnh đầu vào.  
  - FLOPs thấp đồng nghĩa với tốc độ suy luận nhanh và tiết kiệm năng lượng.

- **Inference Latency**  
  - Thời gian mô hình cần để xử lý một ảnh đầu vào (ms).  
  - Đo trên phần cứng mục tiêu như CPU di động hoặc GPU nhúng.